In [1]:
import cameo
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model
from cobra.io import save_json_model
from cameo.flux_analysis.simulation import pfba
import cobra.test
import os
from Functions_Modules.curation_tools import *
from IPython.display import display, HTML

In [2]:
relative_directory = os.getcwd()
filename = relative_directory + '/Reconstructions/MethylococcusModel9.json'
model = cameo.load_model(filename)

Check for funktioning FBA after import as .json File:  

In [3]:
model.objective.variables

{0 <= BIOMASS_REACTION_reverse_42726 <= 0, 0.1 <= BIOMASS_REACTION <= 1000.0}

In [4]:
solution = show_uptake_excretion(model,model.reactions.get_by_id('BIOMASS_REACTION'))

EX_co2_c 9.73776817276
EX_h2o_c 29.7680791941
EX_pi_c -0.0742559453479
EX_hco3_c 2.37162204412
EX_o2_c -27.6123082969
EX_so4_c -0.0273340396411
EX_h2o2_c 0.0010884480203
EX_co_c 4.66477722983e-06
EX_2dhp_c 0.045398233971
EX_5prdmbz_c 0.000155492574328
EX_no3_e -1.3686798476
EX_ch4_p 18.405
EX_fe2_p 0.000951614554886
BIOMASS_REACTION 0.155492574328


In [5]:
model.reactions.pMMO_im.notes

{u'COFACTOR': [u'2 Cu2+', u'1-2 Fe'],
 u'CONFIDENCE SCORE': u'4',
 u'INHIBITOR': u'',
 u'LOCALIZATION': u'Inner Membrane'}

In [6]:
model.reactions.pMMO_im.annotation

{u'brenda': u'1.14.18.3'}

In [7]:
len(model.metabolites)

807

In [8]:
len(model.genes)

654

In [9]:
len(model.reactions)

1488

In [10]:
model.reactions.EX_no3_e.lower_bound = 0

In [11]:
model.reactions.EX_nh3_c.lower_bound = -1000

Construct a function that sets the input parameters of the model for this validation in accordance to Table 1. in [Leak and Dalton's paper, 1986](http://link.springer.com/article/10.1007/BF02346062)

O2-Limiting condition:

"total flow rate of 260ml-min -I. This comprised of the following gases: CH4 (21.9%), N2 (60.4%), O2 (16.9%) and CO2 (0.007%) as"

"0.1 h -1 the cell density was 1.5 g dry wt. 1- ~."

In [12]:
# Calculating the flow rates in mmol/h

ch4_L_per_h = ((0.219*260)/1000)*60
o2_L_per_h = ((0.169*260)/1000)*60
co2_L_per_h = ((0.007*260)/1000)*60

T = 45 + 273.15
R = 0.08216
P = 1

# PV=nRT

ch4_mol_per_L_per_h = (P*ch4_L_per_h)/(R*T)
o2_mol_per_L_per_h = (P*o2_L_per_h)/(R*T)
co2_mol_per_L_per_h = (P*co2_L_per_h)/(R*T)

Olim_ch4_mmol_per_L_per_h = ch4_mol_per_L_per_h*1000
Olim_o2_mmol_per_L_per_h = o2_mol_per_L_per_h*1000
Olim_co2_mmol_per_L_per_h = co2_mol_per_L_per_h*1000

In [19]:
Olim_co2_mmol_per_L_per_h

4.177632953168735

In [13]:
print Olim_ch4_mmol_per_L_per_h,Olim_o2_mmol_per_L_per_h,Olim_co2_mmol_per_L_per_h

130.700230963 100.859995584 4.17763295317


CH4-limiting condition:
    
"total gas flow rate was 430 ml.min -1 and comprised of: CH4 (11%), N2 (68.9%), O2 (19.7%) and CO2 (0.005%)."

In [20]:
# Calculating the flow rates in mmol/h

ch4_L_per_h = ((0.11*430)/1000)*60
o2_L_per_h = ((0.197*430)/1000)*60
co2_L_per_h = ((0.005*430)/1000)*60

T = 45 + 273.15
R = 0.08216
P = 1

# PV=nRT

ch4_mol_per_L_per_h = (P*ch4_L_per_h)/(R*T)
o2_mol_per_L_per_h = (P*o2_L_per_h)/(R*T)
co2_mol_per_L_per_h = (P*co2_L_per_h)/(R*T)

Clim_ch4_mmol_per_L_per_h = ch4_mol_per_L_per_h*1000
Clim_o2_mmol_per_L_per_h = o2_mol_per_L_per_h*1000
Clim_co2_mmol_per_L_per_h = co2_mol_per_L_per_h*1000

In [24]:
print Clim_ch4_mmol_per_L_per_h,Clim_o2_mmol_per_L_per_h,Clim_co2_mmol_per_L_per_h

108.572548728 194.44356454 4.93511585127


Deactivation of NAD+-linked formaldehyde dehydrogenase, since it is only thought to have a detoxifying function with low expression.

In [16]:
model.reactions.ALDD1_c.lower_bound = 0
model.reactions.ALDD1_c.upper_bound = 0

Fix up of gene names:

In [17]:
model.reactions.FALDDH_im.gene_reaction_rule = 'MCA2155'

Define a function that implements the experimental conditions measured by Leak & Dalton in the model:

In [18]:
model.reactions.FALDDH_im.lower_bound = 0
model.reactions.FALDDH_im.upper_bound = 1000

In [19]:
def efficiency_prediction(nitrogen_source,limitation,ratio_pMMO2sMMO,growth_condition,model):
    o2_utr = []
    ch4_utr = []
    grwth_rate = []
    
    for l in range(len(nitrogen_source)):
        
        constraint = model.add_ratio_constraint('pMMO_im','sMMO_c',ratio_pMMO2sMMO[l])
        constraint2 = model.add_ratio_constraint('FALDDH_im','FMFRD_c',ratio_pMMO2sMMO[l])
        
        if nitrogen_source[l] == 'no3':
            model.reactions.EX_nh3_c.lower_bound = 0
            model.reactions.EX_no3_e.lower_bound = -1000
            model.reactions.GLNS_c.upper_bound = 1000
            model.reactions.GLNS_c.lower_bound = 0
        
        elif nitrogen_source[l] == 'nh4':
            model.reactions.EX_nh3_c.lower_bound = -1000
            model.reactions.EX_no3_e.lower_bound = 0
            model.reactions.GLNS_c.knock_out()
        
        if limitation[l] == 'O2':
            model.reactions.EX_o2_c.lower_bound = -Olim_o2_mmol_per_L_per_h
            model.reactions.EX_ch4_p.upper_bound = Olim_ch4_mmol_per_L_per_h
            model.reactions.EX_co2_c.lower_bound = -Olim_co2_mmol_per_L_per_h
            
        elif limitation[l] == 'CH4':
            model.reactions.EX_o2_c.lower_bound = -Clim_o2_mmol_per_L_per_h
            model.reactions.EX_ch4_p.upper_bound = Clim_ch4_mmol_per_L_per_h
            model.reactions.EX_co2_c.lower_bound = -Clim_co2_mmol_per_L_per_h
            
        if growth_condition == 'Chemostat':
            model.reactions.BIOMASS_REACTION.lower_bound = 0.1
            model.reactions.BIOMASS_REACTION.upper_bound = 0.1
            
            try:
                solution = cameo.pfba(model)
                o2_utr.append(solution.fluxes['EX_o2_c'])
                ch4_utr.append(solution.fluxes['EX_ch4_p'])
                grwth_rate.append(solution.fluxes['BIOMASS_REACTION'])
            except cameo.exceptions.SolveError:
                o2_utr.append(0)
                ch4_utr.append(0)
            
        elif growth_condition == 'Batch':
            model.reactions.BIOMASS_REACTION.lower_bound = 0
            model.reactions.BIOMASS_REACTION.upper_bound = 1000

            model.objective.direction = 'max'
            model.change_objective(model.reactions.BIOMASS_REACTION)
        
            try:
                solution = model.solve()
                o2_utr.append(solution.fluxes['EX_o2_c'])
                ch4_utr.append(solution.fluxes['EX_ch4_p'])
                grwth_rate.append(solution.f)
            except cameo.exceptions.SolveError:
                o2_utr.append(0)
                ch4_utr.append(0)
        

        model.solver.remove([constraint,constraint2])
        model.solver.update()
        
    return o2_utr, ch4_utr, grwth_rate

In [20]:
n_sources = ['no3','no3','no3','no3','nh4','nh4','nh4']
limitation = ['O2','O2','CH4','CH4','O2','O2','CH4']
mmo_ratios = [0.02,1,0.01,0.99,0.02,0.99,0.96]

BATCH RESULTS

In [21]:
res,ul,ts = efficiency_prediction(n_sources,limitation,mmo_ratios,'Batch',model)

In [22]:
ratios = [abs(float(res[i]/ul[i])) for i in range(len(res))]
yields = [ts[i]/((ul[i]/1000)*16.04) for i in range(len(res))]

In [23]:
import pandas as pd

In [24]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
     'Limitation' : pd.Series(limitation, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(mmo_ratios, index=range(len(n_sources))),
    'O2/CH4' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)
df

,Limitation,Nitrogen Source,O2/CH4,P/S (%),YCH4
0,O2,no3,1.500261,0.02,0.526707
1,O2,no3,1.500261,1.00,0.526707
2,CH4,no3,1.500261,0.01,0.526707
3,CH4,no3,1.500261,0.99,0.526707
4,O2,nh4,1.500373,0.02,0.752027
5,O2,nh4,1.500373,0.99,0.752027
6,CH4,nh4,1.500373,0.96,0.752027


CHEMOSTAT RESULTS

In [25]:
res,ul,ts = efficiency_prediction(n_sources,limitation,mmo_ratios,'Chemostat',model)

In [26]:
ratios = [abs(float(res[i]/ul[i])) for i in range(len(res))]
yields = [ts[i]/((ul[i]/1000)*16.04) for i in range(len(res))]

In [27]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
     'Limitation' : pd.Series(limitation, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(mmo_ratios, index=range(len(n_sources))),
    'O2/CH4' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)
df

,Limitation,Nitrogen Source,O2/CH4,P/S (%),YCH4
0,O2,no3,1.500261,0.02,0.526707
1,O2,no3,1.500261,1.00,0.526063
2,CH4,no3,1.500261,0.01,0.526707
3,CH4,no3,1.500261,0.99,0.526063
4,O2,nh4,1.500373,0.02,0.752027
5,O2,nh4,1.500373,0.99,0.752027
6,CH4,nh4,1.500373,0.96,0.752027


These results don't correspond to the measurements done by Leak and Dalton. On the basis of previously obtained PSORT data for the enzymes I relocated the Nitrate Reduction pathways and added proper transporters.

In [28]:
model.reactions.HAO_im.id = 'HAO_p'

In [29]:
model.reactions.HAO_p.notes['LOCALIZATION'] = 'Periplasm'

In [30]:
model.reactions.HAO_p.notes

{u'COFACTOR': u'1 Fe',
 u'CONFIDENCE SCORE': u'4',
 u'INHIBITOR': u'',
 u'LOCALIZATION': 'Periplasm'}

In [31]:
model.reactions.CYP460_im.notes['LOCALIZATION'] = 'Periplasm'
model.reactions.CYP460_im.id = 'CYP460_p'

The following cell throws an error, which is a bug according to Joao

In [32]:
model.reactions.NO3R1_im.notes['LOCALIZATION'] = 'Cytosol'
model.reactions.NO3R1_im.id = 'NO3R1_c'
model.reactions.NO3R1_c.add_metabolites({model.metabolites.no2_p: 0,
                     model.metabolites.no2_c: 1,
                     model.metabolites.no3_c: -1,
                     model.metabolites.h2o_c: 1,
                     model.metabolites.no3_p: 0,
                     model.metabolites.h2o_p: 0,
                     model.metabolites.q8h2_im: -1,
                     model.metabolites.q8_im: 1},combine=False)

In [33]:
model.reactions.NOR_im.notes['LOCALIZATION'] = 'Periplasm'

In [34]:
model.reactions.NITR_AM_im.gene_reaction_rule = 'MCA2059'

In [35]:
model.reactions.NITR_NO_im.gene_reaction_rule = 'MCA2059'

In [36]:
model.reactions.NITR_AM_im.add_metabolites({model.metabolites.no2_p: 0,
        model.metabolites.nh3_p: 0,
        model.metabolites.focytcc553_p: -6,
        model.metabolites.ficytcc553_p: 6,
        model.metabolites.h2o_c: 2,
        model.metabolites.nh3_c: 1,
        model.metabolites.no2_c: -1,
        model.metabolites.h_c: -8,
        model.metabolites.no_c: 0,  
        model.metabolites.h2o_p: 0,
        model.metabolites.h_p: 0},combine=False)

In [37]:
model.reactions.NITR_NO_im.add_metabolites({model.metabolites.no2_p: 0,
        model.metabolites.no_p: 0,
        model.metabolites.focytcc553_p: -1,
        model.metabolites.ficytcc553_p: 1,
        model.metabolites.h2o_p: 0,
        model.metabolites.h2o_c: 1,
        model.metabolites.no2_c: -1,
        model.metabolites.h_c: -2,
        model.metabolites.no_c: 1,
        model.metabolites.h_p: 0},combine=False)

In [38]:
model.reactions.NTRIR2x_c.annotation['brenda'] = '1.7.1.4'

In [39]:
model.reactions.NTRIR2y_c.annotation['brenda'] = '1.7.1.4'

In [40]:
rxn = Reaction('NO3abcpp')
rxn.notes['LOCALIZATION'] = 'Inner Membrane'
rxn.notes['CONFIDENCE SCORE'] = '2'
rxn.annotation['brenda'] = '3.6.3.26'
rxn.annotation['metanetx.reaction'] = 'MNXR19979'
rxn.annotation['rhea'] = ['13181','13182','13183','13184']
rxn.annotation['biocyc'] = 'META:3.6.3.26-RXN'
rxn.name = 'Nitrate transport via ABC system (periplasm)'
rxn.gene_reaction_rule = 'MCA0589'
rxn.add_metabolites({model.metabolites.atp_c: -1,
                    model.metabolites.h2o_c: -1,
                    model.metabolites.no3_p: -1,
                    model.metabolites.adp_c: 1,
                    model.metabolites.h_c: 1,
                    model.metabolites.no3_c: 1,
                    model.metabolites.pi_c: 1},combine=False)
rxn.id = 'NO3abcpp_im'
model.add_reaction(rxn)

In [41]:
rxn = Reaction('NO3t7pp')
rxn.notes['LOCALIZATION'] = 'Inner Membrane'
rxn.notes['CONFIDENCE SCORE'] = '2'
rxn.annotation['metanetx.reaction'] = 'MNXR3847'
rxn.annotation['biocyc'] = 'META:TRANS-RXN0-239'
rxn.name = 'Nitrate transport in via nitrite antiport (periplasm)'
rxn.gene_reaction_rule = 'MCA0594'
rxn.add_metabolites({model.metabolites.no2_p: 1,
                    model.metabolites.no2_c: -1,
                    model.metabolites.no3_p: -1,
                    model.metabolites.no3_c: 1},combine=False)
rxn.upper_bound = 1000
rxn.lower_bound = -1000
rxn.id = 'NO3t7pp_im'
model.add_reaction(rxn)

In [42]:
model.repair()

In [43]:
model.reactions.EX_nh3_c.lower_bound = 0
model.reactions.EX_no3_e.lower_bound = -1000

Fixing reactions that potentially bypass the experimentally determined nh3 assimilation pathways:

- High NH3 in medium = Alanine Dehydrogenase
- Low NH3 in medium = GS/GOGAT + Alanine Dehydrogenase

In [44]:
for rxn in model.metabolites.nh3_c.reactions:
    print rxn

DHPPDA_c: 25dhpp_c + h2o_c + h_c --> 5apru_c + nh3_c
GCCb_c: dhlpro_c + mlthf_c + nh3_c --> alpro_c + thf_c
THRD__L_c: thr__L_c --> 2obut_c + nh3_c
AMOs_c: nadh_c + nh3_c + o2_c --> h2o_c + ham_c + nad_c
CYSLYSL_c: Lcystin_c + h2o_c --> nh3_c + pyr_c + thcys_c
ASNS2_c: asp__L_c + atp_c + nh3_c --> amp_c + asn__L_c + h_c + ppi_c
GMPS_c: atp_c + nh3_c + xmp_c --> amp_c + gmp_c + 2.0 h_c + ppi_c
DCTPD2_c: ctp_c + h2o_c + h_c --> nh3_c + utp_c
GLYCL_c: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh3_c
ALAD__L_c: h_c + nadh_c + nh3_c + pyr_c --> ala__L_c + h2o_c + nad_c
HMBS_c: h2o_c + 4.0 ppbng_c <=> hmbil_c + 4.0 nh3_c
CYSTL_c: cyst__L_c + h2o_c --> hcys__L_c + nh3_c + pyr_c
NTRLASE_c: 2.0 h2o_c + ind3acnl_c --> ind3ac_c + nh3_c
GLUDy_c: akg_c + h_c + nadph_c + nh3_c --> glu__L_c + h2o_c + nadp_c
NH3_im: nh3_p <=> nh3_c
NNAM_c: h2o_c + ncam_c --> nac_c + nh3_c
NTRIR2x_c: 5.0 h_c + 3.0 nadh_c + no2_c --> 2.0 h2o_c + 3.0 nad_c + nh3_c
DORNOp_c: 5a2opntn_c + h2o2_c + nh3_c --> h2o_c

In [45]:
model.reactions.HMBS_c.lower_bound = 0

In [46]:
invert_reaction(model.reactions.SERD__L_c)

In [47]:
model.reactions.SERD__L_c.lower_bound = 0

In [48]:
model.remove_reactions([model.reactions.LLEUDr_c])

In [49]:
model.remove_reactions([model.reactions.GLUN_c])

Setting the 2 key ED pathway enzymes irreversible.

In [50]:
invert_reaction(model.reactions.EDD_c)

In [51]:
invert_reaction(model.reactions.EDA_c)

'There is supporting evidence that the serine pathway is connected to the RuBP pathway (20), since phosphoglyco- late could be generated by the oxygenase activity of Rubisco, and further converted by phosphoglycolate phosphatase (L/ H ? 1.62) to glycolate, which is then oxidized to glyoxylate by glyoxylate reductase (L/H ? 0.51).'
- Kao, W.-C., Chen, Y.-R., Yi, E. C., Lee, H., Tian, Q., Wu, K.-M., … Chan, S. I. (2004). Quantitative proteomic analysis of metabolic regulation by copper ions in Methylococcus capsulatus (Bath). The Journal of Biological Chemistry, 279(49), 51554–51560. doi:10.1074/jbc.M408013200

In [52]:
invert_reaction(model.reactions.RBCh_c)

Fixing the TCA cycle:

"The genes for the E1 and E2 subunits of 2-oxoglutarate dehydrogenase do occur in the genome (MCA1952, MCA1953), but it is well established that no activity of this enzyme can be detected in vivo, and without it the Krebs cycle cannot function [6]. Conditions under which the enzyme might be expressed cannot currently be envisaged." 
- Kelly, D. P., Anthony, C., & Murrel, J. C. (2005). Insights into the obligate methanotroph Methylococcus capsulatus. Trends in Microbiology, 13(5), 195–198. doi:10.1016/j.tim.2005.03.002



"Nitrosomonas europaea and Methylococcus capsulatus, which
lack the functional enzyme, were found to contain the coding sequences for the E1 and E2 subunits of a-ketoglutarate dehydrogenase.
Comparing the predicted physicochemical properties of the polypeptides coded by the genes confirmed the putative gene
products were similar to the active a-ketoglutarate dehydrogenase subunits of heterotrophs. These obligate species are thus genomically
competent with respect to this enzyme but are apparently incapable of producing a functional enzyme." 
- Wood, A. P., Aurikko, J. P., & Kelly, D. P. (2004). A challenge for 21st century molecular biology and biochemistry: What are the causes of obligate autotrophy and methanotrophy? FEMS Microbiology Reviews, 28(3), 335–352. doi:10.1016/j.femsre.2003.12.001

In [53]:
for rxn in model.genes.MCA1952.reactions:
    print rxn
    rxn.lower_bound = 0
    rxn.upper_bound = 0
    print rxn

2OXOADOXm_c: 2oxoadp_c + coa_c + nad_c --> co2_c + glutcoa_c + nadh_c
2OXOADOXm_c: 2oxoadp_c + coa_c + nad_c --> co2_c + glutcoa_c + nadh_c
OOR3r_c: co2_c + fdxrd_c + succoa_c <=> akg_c + coa_c + fdxox_c + h_c
OOR3r_c: co2_c + fdxrd_c + succoa_c --> akg_c + coa_c + fdxox_c + h_c
AKGDH_c: akg_c + coa_c + nad_c <=> co2_c + nadh_c + succoa_c
AKGDH_c: akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c


In [54]:
for rxn in model.genes.MCA1953.reactions:
    print rxn
    rxn.lower_bound = 0
    rxn.upper_bound = 0
    print rxn

AKGDH_c: akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
AKGDH_c: akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
OOR3r_c: co2_c + fdxrd_c + succoa_c --> akg_c + coa_c + fdxox_c + h_c
OOR3r_c: co2_c + fdxrd_c + succoa_c --> akg_c + coa_c + fdxox_c + h_c


In [55]:
model.reactions.AKGDH_c.gene_reaction_rule = 'MCA1952 and MCA1953'

In [56]:
model.reactions.OOR3r_c.gene_reaction_rule = 'MCA1952 and MCA1953'

In [57]:
model.remove_reactions([model.reactions.get_by_id('2OXOADOXm_c')])

The model should be able to grow with a knocked out GLNS_c / GS:

In [58]:
model.reactions.GLNS_c.upper_bound = 0

In [59]:
for rxn in model.metabolites.gln__L_c.reactions:
    if model.metabolites.gln__L_c in rxn.products:
        print rxn

ANS_c: anth_c + glu__L_c + h_c + pyr_c --> chor_c + gln__L_c
GLNS_c: atp_c + glu__L_c + nh3_c --> adp_c + gln__L_c + h_c + pi_c


In [60]:
invert_reaction(model.reactions.ANS_c)

In [61]:
model.reactions.ANS_c.lower_bound = -1000

In [62]:
model.reactions.ANS_c.upper_bound = 1000

In [63]:
invert_reaction(model.reactions.GLUDy_c)

I will remove the alternative reaction using NH3 instead of glutamine, as reportedly either GS/GOGAT or AlaDH are the key drivers of N-Assimilation in M. caps.

In [64]:
model.remove_reactions([model.reactions.ANS2_c])

AlaDH can reportedly also use NADP as a cofactor.

In [65]:
rxnX = model.reactions.ALAD__L_c.copy()
rxnX.id = 'ALAD__Ly_c'
rxnX.add_metabolites({model.metabolites.nad_c:-1,
                      model.metabolites.nadh_c:1,
                      model.metabolites.nadp_c:1,
                     model.metabolites.nadph_c:-1},combine=True)
print rxnX
model.add_reaction(rxnX)

ALAD__Ly_c: h_c + nadph_c + nh3_c + pyr_c --> ala__L_c + h2o_c + nadp_c


Despite Murell and Dalton 1983 argue that the glutamate synthase (GOGAT) is irreversible, I see no other way to achieve growth in the model under High NH3 conditions other than when the reaction is set to be reversible. Otherwise the model has no means to synthesize glutamine.

De la torre et al have set this reaction to be reversible in their model of Methylomicrobium.

In [66]:
model.reactions.GLUSy_c.lower_bound = -1000

More appropriate constraints for Growth rate and Methane uptake rate

In [67]:
model.reactions.BIOMASS_REACTION.lower_bound = 0

In [68]:
model.reactions.EX_ch4_p.lower_bound = 0.1

Finally it grows on the GS knockout!

In [69]:
solution = show_uptake_excretion(model,model.reactions.get_by_id('BIOMASS_REACTION'))

EX_co2_c 7.1511942
EX_h2o_c 20.2272947
EX_pi_c -0.4363681
EX_ppi_c 0.1943564
EX_hco3_c 1.0017309
EX_o2_c -18.30570545
EX_so4_c -0.017579
EX_h2o2_c 0.000700000000001
EX_co_c 3e-06
EX_pant_R_c 0.0291964
EX_fdxox_c 5.42101086243e-20
EX_rdmbzi_c 0.0001
EX_no3_e -0.880222
EX_ch4_p 12.2017421
EX_fe2_p 0.000612
BIOMASS_REACTION 0.1


CHEMOSTAT RESULTS with improved model

### Redox Arm hypothesis

In [70]:
res,ul,ts = efficiency_prediction(n_sources,limitation,mmo_ratios,'Chemostat',model)

In [71]:
ratios = [abs(float(res[i]/ul[i])) for i in range(len(res))]
yields = [ts[i]/((ul[i]/1000)*16.04) for i in range(len(res))]

In [72]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
     'CuSO4.5H20 (mg l^-1)':pd.Series([0.2,1.5,0.05,1.5,0.1,1.5,1.5],index=range(len(n_sources))),
     'Limitation' : pd.Series(limitation, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(mmo_ratios, index=range(len(n_sources))),
     'O2/CH4 - Experimental' : pd.Series([1.47,1.45,1.50,1.41,1.56,1.60,1.60], index=range(len(n_sources))),
     'YCH4 - Experimental' : pd.Series([0.50,0.69,0.50,0.67,0.71,'0.69-0.75',0.67], index=range(len(n_sources))),
    'O2/CH4 - Predicted' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4 - Predicted' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)

In [73]:
HTML(df.to_html())

,CuSO4.5H20 (mg l^-1),Limitation,Nitrogen Source,O2/CH4 - Experimental,O2/CH4 - Predicted,P/S (%),YCH4 - Experimental,YCH4 - Predicted
0,0.20,O2,no3,1.47,1.502724,0.02,0.5,0.526049
1,1.50,O2,no3,1.45,1.502724,1.00,0.69,0.526049
2,0.05,CH4,no3,1.50,1.502724,0.01,0.5,0.526049
3,1.50,CH4,no3,1.41,1.502724,0.99,0.67,0.526049
4,0.10,O2,nh4,1.56,1.503876,0.02,0.71,0.748384
5,1.50,O2,nh4,1.60,1.503876,0.99,0.69-0.75,0.748384
6,1.50,CH4,nh4,1.60,1.503876,0.96,0.67,0.748384


The model is predictive of the Nitrogen source but it does not reflect the fractions of pMMO vs sMMO.

### Uphill electron Transfer!

In [74]:
model.reactions.CYOR_q8_im.lower_bound = -1.7 #-1.7

In [75]:
res,ul,ts = efficiency_prediction(n_sources,limitation,mmo_ratios,'Chemostat',model)

In [76]:
res,ul,ts

([-15.88815718725487,
  -14.409366650000067,
  -15.967744049999824,
  -14.409366650000166,
  -12.367269187254882,
  -9.12803464999994,
  -9.128034649999943],
 [10.890780568627427,
  10.151385300000042,
  10.930573999999877,
  10.151385300000108,
  8.250114568627437,
  6.630497299999961,
  6.630497299999962],
 [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [77]:
ratios = [abs(float(res[i]/ul[i])) for i in range(len(res))]
yields = [ts[i]/((ul[i]/1000)*16.04) for i in range(len(res))]

In [78]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
     'CuSO4.5H20 (mg l^-1)':pd.Series([0.2,1.5,0.05,1.5,0.1,1.5,1.5],index=range(len(n_sources))),
     'Limitation' : pd.Series(limitation, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(mmo_ratios, index=range(len(n_sources))),
     'O2/CH4 - Experimental' : pd.Series([1.47,1.45,1.50,1.41,1.56,1.60,1.60], index=range(len(n_sources))),
     'YCH4 - Experimental' : pd.Series([0.50,0.69,0.50,0.67,0.71,'0.69-0.75',0.67], index=range(len(n_sources))),
    'O2/CH4 - Predicted' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4 - Predicted' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)

In [79]:
HTML(df.to_html())

,CuSO4.5H20 (mg l^-1),Limitation,Nitrogen Source,O2/CH4 - Experimental,O2/CH4 - Predicted,P/S (%),YCH4 - Experimental,YCH4 - Predicted
0,0.20,O2,no3,1.47,1.458863,0.02,0.5,0.572449
1,1.50,O2,no3,1.45,1.419448,1.00,0.69,0.614144
2,0.05,CH4,no3,1.50,1.460833,0.01,0.5,0.570365
3,1.50,CH4,no3,1.41,1.419448,0.99,0.67,0.614144
4,0.10,O2,nh4,1.56,1.499042,0.02,0.71,0.755676
5,1.50,O2,nh4,1.60,1.376674,0.99,0.69-0.75,0.940263
6,1.50,CH4,nh4,1.60,1.376674,0.96,0.67,0.940263


The above is a lot closer to the experimentally measured values.

### Direct coupling!

In [80]:
model.reactions.CYOR_q8_im.lower_bound = 0

In [81]:
pMMO_dir_coupl_im = model.reactions.pMMO_im.copy()

In [82]:
model.reactions.pMMO_im.upper_bound = 0

In [83]:
pMMO_dir_coupl_im.id = 'pMMO_dir_coupl_im'
pMMO_dir_coupl_im.add_metabolites({model.metabolites.q8h2_im: 1,
                              model.metabolites.q8_im: -1,
                              model.metabolites.focytcc555_p: -2,
                              model.metabolites.ficytcc555_p: 2},combine = True)
model.add_reaction(pMMO_dir_coupl_im)

In [84]:
model.reactions.pMMO_dir_coupl_im

Id,pMMO_dir_coupl_im
Name,particulate Methane Monooxygenase
Stoichiometry,ch4_p + 2 focytcc555_p + o2_p --> 2 ficytcc555_p + h2o_p + meoh_p
GPR,(((MCA1796 and MCA1797 and MCA1798) or (MCA2853 and MCA2854 and MCA2855)) and MCA0295)
Lower bound,0.000000
Upper bound,1000.000000


In [85]:
res,ul,ts = efficiency_prediction(n_sources,limitation,mmo_ratios,'Chemostat',model)

In [86]:
ratios = [abs(float(res[i]/ul[i])) for i in range(len(res))]
yields = [ts[i]/((ul[i]/1000)*16.04) for i in range(len(res))]

In [87]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
     'CuSO4.5H20 (mg l^-1)':pd.Series([0.2,1.5,0.05,1.5,0.1,1.5,1.5],index=range(len(n_sources))),
     'Limitation' : pd.Series(limitation, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(mmo_ratios, index=range(len(n_sources))),
     'O2/CH4 - Experimental' : pd.Series([1.47,1.45,1.50,1.41,1.56,1.60,1.60], index=range(len(n_sources))),
     'YCH4 - Experimental' : pd.Series([0.50,0.69,0.50,0.67,0.71,'0.69-0.75',0.67], index=range(len(n_sources))),
    'O2/CH4 - Predicted' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4 - Predicted' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)

In [88]:
HTML(df.to_html())

,CuSO4.5H20 (mg l^-1),Limitation,Nitrogen Source,O2/CH4 - Experimental,O2/CH4 - Predicted,P/S (%),YCH4 - Experimental,YCH4 - Predicted
0,0.20,O2,no3,1.47,1.005449,0.02,0.5,1.052099
1,1.50,O2,no3,1.45,1.005449,1.00,0.69,1.052099
2,0.05,CH4,no3,1.50,1.005449,0.01,0.5,1.052099
3,1.50,CH4,no3,1.41,1.005449,0.99,0.67,1.052099
4,0.10,O2,nh4,1.56,1.007804,0.02,0.71,1.496690
5,1.50,O2,nh4,1.60,1.007752,0.99,0.69-0.75,1.496769
6,1.50,CH4,nh4,1.60,1.007752,0.96,0.67,1.496769


### Finding the percentage splits of redox-arm, direct-coupling and uphill-electron-transfer

In [89]:
from cameo.util import TimeMachine

In [96]:
with TimeMachine() as tm:
    try:
        constraint = model.add_ratio_constraint('pMMO_im','sMMO_c',0.02)
        constraint2 = model.add_ratio_constraint('FALDDH_im','FMFRD_c',0.02)
        
#         reaction1 = model.reactions.EX_o2_c
#         reaction2 = model.reactions.EX_ch4_p
#         ratio = 1.47
#         flux_ratio_constraint = model.solver.interface.Constraint(reaction1.flux_expression - ratio * reaction2.flux_expression, lb = 1.47)
#         model.solver.add(flux_ratio_constraint)
        
        model.reactions.EX_nh3_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.EX_no3_e.change_bounds(lb=-1000,ub=1000,time_machine=tm)
        model.reactions.GLNS_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.EX_o2_c.change_bounds(lb=-Olim_o2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        model.reactions.EX_ch4_p.change_bounds(lb=0,ub=Olim_ch4_mmol_per_L_per_h,time_machine=tm)
        model.reactions.EX_co2_c.change_bounds(lb=-Olim_co2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        
        model.reactions.BIOMASS_REACTION.change_bounds(lb=0.1,ub=0.1,time_machine=tm)
        model.reactions.NGAM_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        
        model.reactions.pMMO_dir_coupl_im.change_bounds(lb=0,ub=0,time_machine=tm)
        model.reactions.pMMO_im.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.CYOR_q8_im.change_bounds(lb=-0.73019,ub=0,time_machine=tm)
        
        solution = cameo.pfba(model)
        print solution.fluxes['pMMO_dir_coupl_im'],solution.fluxes['pMMO_im'],solution.fluxes['CYOR_q8_im']
        print abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))
        print 1.47-abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))

    finally:
        model.solver.remove([constraint,constraint2])
        model.solver.update()
#         model.solver.remove([flux_ratio_constraint])
#         model.solver.update()

0.0 0.218062652941 -0.73019
1.47007459261
-7.45926114618e-05


In [134]:
with TimeMachine() as tm:
    try:
        constraint = model.add_ratio_constraint('pMMO_im','sMMO_c',1.)
        constraint2 = model.add_ratio_constraint('FALDDH_im','FMFRD_c',1.)
        
#         reaction1 = model.reactions.EX_o2_c
#         reaction2 = model.reactions.EX_ch4_p
#         ratio = 1.47
#         flux_ratio_constraint = model.solver.interface.Constraint(reaction1.flux_expression - ratio * reaction2.flux_expression, lb = 1.47)
#         model.solver.add(flux_ratio_constraint)
        
        model.reactions.EX_nh3_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.EX_no3_e.change_bounds(lb=-1000,ub=1000,time_machine=tm)
        model.reactions.GLNS_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.EX_o2_c.change_bounds(lb=-Olim_o2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        model.reactions.EX_ch4_p.change_bounds(lb=0,ub=Olim_ch4_mmol_per_L_per_h,time_machine=tm)
        model.reactions.EX_co2_c.change_bounds(lb=-Olim_co2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        
        model.reactions.BIOMASS_REACTION.change_bounds(lb=0.1,ub=0.1,time_machine=tm)
        model.reactions.NGAM_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        
        model.reactions.pMMO_dir_coupl_im.change_bounds(lb=0,ub=0,time_machine=tm)
        model.reactions.pMMO_im.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.CYOR_q8_im.change_bounds(lb=-1.12999,ub=0,time_machine=tm)
        
        solution = cameo.pfba(model)
        print solution.fluxes['pMMO_dir_coupl_im'],solution.fluxes['pMMO_im'],solution.fluxes['CYOR_q8_im']
        print abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))
        print 1.45-abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))

    finally:
        model.solver.remove([constraint,constraint2])
        model.solver.update()
#         model.solver.remove([flux_ratio_constraint])
#         model.solver.update()

0.0 5.36069765 -1.12999
1.45031371523
-0.000313715230704


In [185]:
with TimeMachine() as tm:
    try:
        constraint = model.add_ratio_constraint('pMMO_im','sMMO_c',0.01)
        constraint2 = model.add_ratio_constraint('FALDDH_im','FMFRD_c',0.01)
        
#         reaction1 = model.reactions.EX_o2_c
#         reaction2 = model.reactions.EX_ch4_p
#         ratio = 1.47
#         flux_ratio_constraint = model.solver.interface.Constraint(reaction1.flux_expression - ratio * reaction2.flux_expression, lb = 1.47)
#         model.solver.add(flux_ratio_constraint)
        
        model.reactions.EX_nh3_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.EX_no3_e.change_bounds(lb=-1000,ub=1000,time_machine=tm)
        model.reactions.GLNS_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.EX_o2_c.change_bounds(lb=-Olim_o2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        model.reactions.EX_ch4_p.change_bounds(lb=0,ub=Olim_ch4_mmol_per_L_per_h,time_machine=tm)
        model.reactions.EX_co2_c.change_bounds(lb=-Olim_co2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        
        model.reactions.BIOMASS_REACTION.change_bounds(lb=0.1,ub=0.1,time_machine=tm)
        model.reactions.NGAM_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        
        model.reactions.pMMO_dir_coupl_im.change_bounds(lb=0,ub=0.063,time_machine=tm)
        model.reactions.pMMO_im.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.CYOR_q8_im.change_bounds(lb=0,ub=0,time_machine=tm)
        
        solution = cameo.pfba(model)
        print solution.fluxes['pMMO_dir_coupl_im'],solution.fluxes['pMMO_im'],solution.fluxes['CYOR_q8_im']
        print abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))
        print 1.50-abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))

    finally:
        model.solver.remove([constraint,constraint2])
        model.solver.update()
#         model.solver.remove([flux_ratio_constraint])
#         model.solver.update()

0.063 0.116092923762 0.0
1.50006690484
-6.69048372555e-05


In [186]:
with TimeMachine() as tm:
    try:
        constraint = model.add_ratio_constraint('pMMO_im','sMMO_c',0.99)
        constraint2 = model.add_ratio_constraint('FALDDH_im','FMFRD_c',0.99)
        
#         reaction1 = model.reactions.EX_o2_c
#         reaction2 = model.reactions.EX_ch4_p
#         ratio = 1.47
#         flux_ratio_constraint = model.solver.interface.Constraint(reaction1.flux_expression - ratio * reaction2.flux_expression, lb = 1.47)
#         model.solver.add(flux_ratio_constraint)
        
        model.reactions.EX_nh3_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.EX_no3_e.change_bounds(lb=-1000,ub=1000,time_machine=tm)
        model.reactions.GLNS_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.EX_o2_c.change_bounds(lb=-Olim_o2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        model.reactions.EX_ch4_p.change_bounds(lb=0,ub=Olim_ch4_mmol_per_L_per_h,time_machine=tm)
        model.reactions.EX_co2_c.change_bounds(lb=-Olim_co2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        
        model.reactions.BIOMASS_REACTION.change_bounds(lb=0.1,ub=0.1,time_machine=tm)
        model.reactions.NGAM_c.change_bounds(lb=0,ub=1000,time_machine=tm)
        
        model.reactions.pMMO_dir_coupl_im.change_bounds(lb=0,ub=0,time_machine=tm)
        model.reactions.pMMO_im.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.CYOR_q8_im.change_bounds(lb=-1.86,ub=0,time_machine=tm)
        
        solution = cameo.pfba(model)
        print solution.fluxes['pMMO_dir_coupl_im'],solution.fluxes['pMMO_im'],solution.fluxes['CYOR_q8_im']
        print abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))
        print 1.41-abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))

    finally:
        model.solver.remove([constraint,constraint2])
        model.solver.update()
#         model.solver.remove([flux_ratio_constraint])
#         model.solver.update()

0.0 4.97058866683 -1.86
1.41015146819
-0.000151468185298


In [194]:
with TimeMachine() as tm:
    try:
        constraint = model.add_ratio_constraint('pMMO_im','sMMO_c',0.02)
        constraint2 = model.add_ratio_constraint('FALDDH_im','FMFRD_c',0.02)
        
#         reaction1 = model.reactions.EX_o2_c
#         reaction2 = model.reactions.EX_ch4_p
#         ratio = 1.47
#         flux_ratio_constraint = model.solver.interface.Constraint(reaction1.flux_expression - ratio * reaction2.flux_expression, lb = 1.47)
#         model.solver.add(flux_ratio_constraint)
        
        model.reactions.EX_nh3_c.change_bounds(lb=-1000,ub=1000,time_machine=tm)
        model.reactions.EX_no3_e.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.GLNS_c.change_bounds(lb=0,ub=0,time_machine=tm)
        model.reactions.EX_o2_c.change_bounds(lb=-Olim_o2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        model.reactions.EX_ch4_p.change_bounds(lb=0,ub=Olim_ch4_mmol_per_L_per_h,time_machine=tm)
        model.reactions.EX_co2_c.change_bounds(lb=-Olim_co2_mmol_per_L_per_h,ub=1000,time_machine=tm)
        
        model.reactions.BIOMASS_REACTION.change_bounds(lb=0.1,ub=0.1,time_machine=tm)
        model.reactions.NGAM_c.change_bounds(lb=60,ub=1000,time_machine=tm)
        
        model.reactions.pMMO_dir_coupl_im.change_bounds(lb=0,ub=0,time_machine=tm)
        model.reactions.pMMO_im.change_bounds(lb=0,ub=1000,time_machine=tm)
        model.reactions.CYOR_q8_im.change_bounds(lb=0.0,ub=0,time_machine=tm)
        
        solution = cameo.pfba(model)
        print solution.fluxes['pMMO_dir_coupl_im'],solution.fluxes['pMMO_im'],solution.fluxes['CYOR_q8_im']
        print solution.fluxes['EX_o2_c'],solution.fluxes['EX_ch4_p']
        print abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))
        print 1.56-abs(float(solution.fluxes['EX_o2_c']/solution.fluxes['EX_ch4_p']))

    finally:
        model.solver.remove([constraint,constraint2])
        model.solver.update()
#         model.solver.remove([flux_ratio_constraint])
#         model.solver.update()

0.0 0.163343084314 0.0
-12.52803465 8.3304973
1.50387596308
0.0561240369167


Using the fit parameter

In [210]:
with TimeMachine() as tm:
    model.reactions.pMMO_dir_coupl_im.change_bounds(lb=0,ub=0,time_machine=tm)
    model.reactions.pMMO_im.change_bounds(lb=0,ub=1000,time_machine=tm)
    model.reactions.CYOR_q8_im.change_bounds(lb=-1.88,ub=0,time_machine=tm)
    res,ul,ts = efficiency_prediction(n_sources,limitation,mmo_ratios,'Chemostat',model)

ratios = [abs(float(res[i]/ul[i])) for i in range(len(res))]
yields = [ts[i]/((ul[i]/1000)*16.04) for i in range(len(res))]

d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
     'CuSO4.5H20 (mg l^-1)':pd.Series([0.2,1.5,0.05,1.5,0.1,1.5,1.5],index=range(len(n_sources))),
     'Limitation' : pd.Series(limitation, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(mmo_ratios, index=range(len(n_sources))),
     'O2/CH4 - Experimental' : pd.Series([1.47,1.45,1.50,1.41,1.56,1.60,1.60], index=range(len(n_sources))),
     'YCH4 - Experimental' : pd.Series([0.50,0.69,0.50,0.67,0.71,'0.69-0.75',0.67], index=range(len(n_sources))),
    'O2/CH4 - Predicted' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4 - Predicted' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)

HTML(df.to_html())

,CuSO4.5H20 (mg l^-1),Limitation,Nitrogen Source,O2/CH4 - Experimental,O2/CH4 - Predicted,P/S (%),YCH4 - Experimental,YCH4 - Predicted
0,0.20,O2,no3,1.47,1.458863,0.02,0.5,0.572449
1,1.50,O2,no3,1.45,1.408968,1.00,0.69,0.625230
2,0.05,CH4,no3,1.50,1.460833,0.01,0.5,0.570365
3,1.50,CH4,no3,1.41,1.408968,0.99,0.67,0.625230
4,0.10,O2,nh4,1.56,1.499042,0.02,0.71,0.755676
5,1.50,O2,nh4,1.60,1.359280,0.99,0.69-0.75,0.966501
6,1.50,CH4,nh4,1.60,1.359280,0.96,0.67,0.966501


In [211]:
df.to_csv('PartialValidation.csv')

In [212]:
target_filename_json = relative_directory + '/Reconstructions/MethylococcusModel10.json'
target_filename_xml = relative_directory + '/Reconstructions/MethylococcusModel10.xml'
cobra.io.write_sbml_model(model, target_filename_xml, use_fbc_package=True)
cobra.io.save_json_model(model, target_filename_json,pretty = True)

In [ ]:
#model.reactions.pMMO_dir_coupl_im.upper_bound = 0

In [213]:
import escher
        
maps = escher.Builder(
                    model=model,
                    map_name = 'Methylococcus capsulatus genome-scale metabolic model',
                    map_json='/Users/clie/Desktop/EFPro2/Reconstructions/Metabolic Map/15MinorChanges.json',
                    reaction_data=solution.fluxes,
                    show_gene_reaction_rules=True,
                    use_3d_transform = True,
                    )

maps.set_reaction_scale([ { 'type': 'min', 'color': 'rgb(235,89,162)', 'size': 12 },
   { 'type': 'median', 'color': 'rgb(200,200,200)', 'size': 20 },
   { 'type': 'max', 'color': 'rgb(255,255,109)', 'size': 25 } ])
maps.display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


/Users/clie/.virtualenvs/modeling/lib/python2.7/site-packages/escher/plots.py:351: UserWarning:

map_json overrides map_name

127.0.0.1 - - [22/Sep/2016 10:32:28] "GET / HTTP/1.1" 200 -



stopping Server...


In [ ]:
try:
    stuff
except Exception as e:
    print(e)

In [ ]:
model.reactions.BIOMASS_REACTION